# OpEn Rust Examples: Various Obstacles - Part 1

In the previous example, we implemented a path planner for a mobile robot to avoid multiple circular obstacles. For now, we are going to address **other shaped obstacles** including **nonlinear-shaped obstacles**. This contents will be divided into two parts: (1) Mathematical formations for nonlinear shaped obstacles; (2) Adding a path planner based on them.   


## Features

- We already learnt about **nonlinear-shaped obstacle** formulation in [Example 06](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_obs_avoidance_simplified.ipynb). However, uptil now, we had used a circular obstacle for simplicity.  

- We use **the numerical gradient function** in [Example 07](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_general_diff.ipynb), which allows you not to need analytically obtain the derivatives/gradients of objective functions that you are using. 



## Obstacle Constraints Formulation

### Preliminary: Basic Form

From [Example 06](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_obs_avoidance_simplified.ipynb), let's remind of that the basic form of obstacle avoidance constraints is as follows:

$$ O = \{ x \in \mathbb{R}^{n_d} : h^i(x) > 0, \forall i \in \mathbb{N}_{[1,m]} \} $$  
<div style="text-align: right"> (1) </div>

where 
- $n_d$: the number of space dimensions (normally 2 or 3)
- $h^i$: the $i$-th nonlinear function that is formed of the obstacle shape. **This function should be designed such that $h^i$ becomes larger than zero as $x$ moves toward inside the obstacle**. 
- $\mathbb{N}_{[1,m]} = \{1,2,...,m\}$: a natural number set from 1 to $m$, which is the number of nonlinear functions for the obstacle.


Let's say $x$ is the position of a mobile robot. In order for the robot to avoid the obstacle, we should make sure the following constraint for every time instant:

$$ x \notin O $$
<div style="text-align: right"> (2) </div>

This constraint is equivalent to 

$$h^i(x) \le 0, \text{for some }i \in \mathbb{N}_{[1,m]} $$
<div style="text-align: right"> (3) </div>

which is reduced to 

$$\prod_{i=1}^{m} [h^i(x)]^2_{+} = 0$$
<div style="text-align: right"> (4) </div>

For now, we are going to have a look the mathematical formulations for different types of obstacles. 

### (1) Elipsoid

$$h(x) = 1-(x-c)^{\top}E(x-c)$$

where 
- $c \in \mathbb{R}^{n_d}$ is the centre of the elipsoid
- $E \in \mathbb{R}^{n_d \times n_d}$ is a diagonal matrix that controls the size of the elipsoid (i.e. $E = diag(\frac{1}{r_1^2},..., \frac{1}{r^2_{n_d}}$) )

As $x$ is close to $c$, $h(x)$ becomes higher. 

#### 2D Implementation

For 2D, it can be implemented as follows:

In [32]:
// x: user position
// elip : (centre_x, centre_y, radius_x, radius_y)
fn h_elip(x: &[f64], elip: (f64, f64, f64, f64)) -> f64{
    
    
    let h = (1.0 - ((x[0]-elip.0)/elip.2).powi(2) - ((x[1]-elip.1)/elip.3).powi(2) ).max(0.0);  
    
    if h > 0.0 {
        println!("x is inside the obstacle");
    }
    else{
        println!("x is outside the obstacle");
    }
    
    return h;
}


### (2) Polyhedral

$$\prod_{i=1}^{m} [h^i(x)]^2_{+} = 0$$

where 
$$h^i(x) = b_i - a_i^{\top} x \quad \text{ for each plane $i$ of the object;} $$ 

- $b_i \in \mathbb{R}$: the plane's bias (i.e. distance) from the origin 
- $a_i \in \mathbb{R}^{n_d}$: the outward normal vector of the plane

#### 2D Implementation

In [84]:
// x: user position
// poly : <(a_i, b_i)>
fn h_poly(x: &[f64], poly: Vec<((f64, f64), f64)>, centre: (f64, f64)) -> f64{
    let mut h: f64 = 1.0;
    for i in 0..poly.len(){
        let a_i = poly[i].0;
        let b_i = poly[i].1;
        h *= (b_i - (a_i.0*(x[0] - centre.0) + a_i.1*(x[1]- centre.1))).max(0.0);        
    }
    
    if h > 0.0 {
        println!("x is inside the obstacle");
    }
    else{
        println!("x is outside the obstacle");
    }
    
    return h;
}

### (3) Non-linear Obstacles

The followings are non-linear obtacles used in [Sathya et al., 2019](https://arxiv.org/pdf/1904.10546.pdf) (See Fig 3 in the paper).

$$O_1 = \{y > x^2, y < 1 + \frac{x^2}{2} \} $$

$$O_2 = \{y > 2 \sin(-\frac{x}{2}), y < 3 \sin(\frac{x}{2}-1), 1 < x < 8 \} $$

For now, we are implementing them with addition of their central positions as arguments so that we can adjust their positions. 


Then, the constraint for $O_1$ becomes

$$\prod_{i=1}^{m} [h^i(x)]^2_{+} = 0$$

where

$$h^1 := (y - c_y) - (x - c_x)^2 > 0$$
$$h^2 := 1 + \frac{(x - c_x)^2}{2} - (y - c_y) > 0$$

In [72]:
fn h_nlr_1(x: &[f64], centre: (f64, f64)) -> f64{
    // x: user position
    // centre : (c_x, c_y)    
    
    let h1 = ((x[1] - centre.1) - ((x[0] - centre.0)).powi(2)).max(0.0);
    let h2 = (1.0 + (x[0] - centre.0).powi(2)/2.0 - (x[1] - centre.1)).max(0.0);
    
    let h = h1*h2;
    
    if h > 0.0 {
        println!("x is inside the obstacle");
    }
    else{
        println!("x is outside the obstacle");
    }
    
    return h;
}

The constraint for $O_2$ becomes

$$\prod_{i=1}^{m} [h^i(x)]^2_{+} = 0$$

where 

$$h_1 := (y - c_y) - 2 \sin(-\frac{x - c_x}{2})$$
$$h_{2-1} := 3\sin(\frac{x-c_x}{2} - 1) - (y-c_y)$$
$$h_{2-2} := x - c_x - 1$$
$$h_{2-3} := 8 - (x - c_x)$$

In [75]:
fn h_nlr_2(x: &[f64], centre: (f64, f64)) -> f64{
    // x: user position
    // centre : (c_x, c_y)    
    
    let h1 = ((x[1] - centre.1) - 2.0*(-(x[0] - centre.0)/2.0).sin()).max(0.0);
    let h2_1 = (3.0*(((x[0] - centre.0)/2.0) - 1.0).sin() - (x[1] - centre.1)).max(0.0);
    let h2_2 = (x[0] - centre.0 - 1.0).max(0.0);
    let h2_3 = (8.0 - (x[0] - centre.0)).max(0.0);
    
    let h = h1*h2_1*h2_2*h2_3;
    
    if h > 0.0 {
        println!("x is inside the obstacle");
    }
    else{
        println!("x is outside the obstacle");
    }
    
    return h;
}

### Test

- Elipsoid: Center $(5, 0)$ and Radius $(2, 1.5)$
- Rectangular: Centre $(0, 0)$, Normal Vectors $\{ (1,0), (-1,0), (0, 1), (0, -1) \}$
- Pentagonal: Centre $(0, 5)$, Normal Vectors $\{ (1,2), (-2,1), (-1, -1), (1, -1), (0, -1) \}$
- Non-linear Object One: Centre $(-5, 0)$
- Non-linear Object Two: Centre $(-5, -5)$

![](https://raw.githubusercontent.com/inmo-jang/optimisation_tutorial/master/tools_examples/OpEn/examples_rust/example_10_nonlinear_obstacles/result.svg)

In [79]:
h_elip(&[3.0, 0.0],(5.0, 0.0, 2.0, 1.5));
h_elip(&[3.01, 0.0],(5.0, 0.0, 2.0, 1.5));

x is outside the obstacle
x is inside the obstacle


In [89]:
let mut poly_A = Vec::new();
poly_A.push(((1.0_f64, 0.0_f64), 1.0_f64));
poly_A.push(((-1.0_f64, 0.0_f64), 1.0_f64));
poly_A.push(((0.0_f64, 1.0_f64), 1.0_f64));
poly_A.push(((0.0_f64, -1.0_f64), 1.0_f64));

In [92]:
h_poly(&[1.0, 1.0], poly_A.clone(), (0.0, 0.0));
h_poly(&[0.999, 0.999], poly_A.clone(), (0.0, 0.0));

x is outside the obstacle
x is inside the obstacle


## Discussion

Given a position, the $h$ functions output whether the point is inside the obstacle or not. 

Soon (i.e. [in the next part](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_nonlinear_obstacles_02.ipynb)), we will integrate the path planner example along with the above obstacles. 
